In [51]:
import numpy as np
from Data_Savior_J import load_file

Xz = load_file("./classifier_data/walk1.data")
Xz = np.vstack((Xz,load_file("./classifier_data/walk1U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk1D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk2.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk3.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk4.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk5.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5D.data")))

## Vetor de features para classificação:

$X_c = [a~| ~av~| ~aa~| ~l\_a~| ~l\_av~| ~l\_aa~| ~pos\_foot\_r~| ~pos\_foot\_l~| ~vz\_r~| ~vz\_l~| ~C]$
$X_c = [0~|~~1~~| ~~2~~| ~~~3~~| ~~~4~~~| ~~~~5~~| ~~~~~~~6~~~~~~~~~~| ~~~~~~~~7~~~~~~~~| ~~~8~~~| ~~~~9~~| 10]$

#### $a \rightarrow$ ângulo do joelho direito; $av \rightarrow$ velocidade angular do joelho direito; $aa \rightarrow$ aceleração angular do joelho direito;
#### $a \rightarrow$ ângulo do joelho esquerdo; $av \rightarrow$ velocidade angular do joelho esquerdo; $aa \rightarrow$ aceleração angular do joelho esquerdo; 
#### $pos\_foot\_r \rightarrow$ posição do tornozelo direito em relação ao sacro; $pos\_foot\_l \rightarrow$ posição do tornozelo esquerdo em relação ao sacro;
#### $vz\_r \rightarrow$ velocidade do trocanter direito no eixo z; $vz\_l \rightarrow$ velocidade do trocanter esquerdo no eixo z;
#### $C \rightarrow$ indice de classificação
## Indice de classificação $"c"$:
#### $C = 0 \rightarrow$ Marcha normal;
#### $C = 1 \rightarrow$ Marcha de subida de escada;
#### $C = 2 \rightarrow$ Marvha de descidade escada.

In [52]:
import numpy as np

X = Xz[:,[0,1,2,3,4,5,6,7,8,9]]

yz = Xz[:,[10]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((817, 10), (817,))

In [53]:
np.unique(y) # possíveis valores de y

array([ 0.,  1.,  2.])

In [54]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=10)

In [55]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


In [56]:
print X_train_std.shape

(612, 10)


# SVM RBF/Polynomial Classifier

In [98]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

#print 'Processing time RBF'
#%time rbf_svc = svm.SVC(kernel='rbf', gamma=0.2, C=6, decision_function_shape='ovr').fit(X_train_std, y_train)
print ''
print 'Processing time Polynomial'
%time poly_svc = svm.SVC(kernel='poly', degree=2, coef0=4.7, C=48.9, decision_function_shape='ovr').fit(X_train_std, y_train)

from sklearn.metrics import accuracy_score
y_pred = poly_svc.predict(X_test_std)

def run_svm(svc, X_test_std, y_test):
    y_pred = svc.predict(X_test_std)
    from sklearn.metrics import accuracy_score
    if (svc==rbf_svc):
        print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
    elif(svc==poly_svc):
        print ('SVM-Polynomial accuracy:--->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
        
run_svm(rbf_svc, X_test_std, y_test)
#run_svm(poly_svc, X_test_std, y_test)


Processing time Polynomial
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.1 ms
SVM-RBF accuracy:---------->99.02 %


# Confusion Matrix for SVM RBF/Polynomial Classifier

In [99]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[62  0  0]
 [ 1 65  1]
 [ 0  0 76]]
Normalized confusion matrix
[[ 1.    0.    0.  ]
 [ 0.01  0.97  0.01]
 [ 0.    0.    1.  ]]


In [100]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 99.00 %

Sensitivity-A: 100.00 %
Sensitivity-B: 97.01 %
Sensitivity-C: 100.00 %

Precision-A: 98.53 %
Precision-B: 100.00 %
Precision-C: 98.53 %

Specificity-A: 99.25 %
Specificity-B: 100.00 %
Specificity-C: 99.25 %


# Gaussian Naive Bayes Classifier

In [83]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_std, y_train)

y_pred = gnb.predict(X_test_std)

from sklearn.metrics import accuracy_score
print ('ClassifyGNB accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

ClassifyGNB accuracy:---------->87.32 %


# Confusion Matrix for Gaussian Naive Bayes Classifier

In [84]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[58  2  2]
 [10 46 11]
 [ 1  0 75]]
Normalized confusion matrix
[[ 0.94  0.03  0.03]
 [ 0.15  0.69  0.16]
 [ 0.01  0.    0.99]]


In [85]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 86.96 %

Sensitivity-A: 93.55 %
Sensitivity-B: 68.66 %
Sensitivity-C: 98.68 %

Precision-A: 85.21 %
Precision-B: 95.51 %
Precision-C: 83.40 %

Specificity-A: 91.88 %
Specificity-B: 96.83 %
Specificity-C: 90.18 %


# Random Forest Classifier

In [86]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

print 'Training time'
%time rfc = rfc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time y_pred = rfc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

Training time
CPU times: user 300 ms, sys: 0 ns, total: 300 ms
Wall time: 296 ms

Predict time
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 39.9 ms

ClassifyRF accuracy:---------->99.51 %


# Confusion Matrix for Random Forest Classifier

In [87]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[62  0  0]
 [ 1 66  0]
 [ 0  0 76]]
Normalized confusion matrix
[[ 1.    0.    0.  ]
 [ 0.01  0.99  0.  ]
 [ 0.    0.    1.  ]]


In [88]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 99.50 %

Sensitivity-A: 100.00 %
Sensitivity-B: 98.51 %
Sensitivity-C: 100.00 %

Precision-A: 98.53 %
Precision-B: 100.00 %
Precision-C: 100.00 %

Specificity-A: 99.25 %
Specificity-B: 100.00 %
Specificity-C: 100.00 %


# Decision Tree Classifier

In [89]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(min_samples_split = 20)
dtc = dtc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time y_pred = dtc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyDT accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))


Predict time
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 173 µs

ClassifyDT accuracy:---------->95.12 %


# Confusion Matrix for Decision Tree Classifier

In [90]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[61  1  0]
 [ 0 62  5]
 [ 1  3 72]]
Normalized confusion matrix
[[ 0.98  0.02  0.  ]
 [ 0.    0.93  0.07]
 [ 0.01  0.04  0.95]]


In [91]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 95.22 %

Sensitivity-A: 96.83 %
Sensitivity-B: 92.54 %
Sensitivity-C: 94.74 %

Precision-A: 98.68 %
Precision-B: 94.33 %
Precision-C: 92.70 %

Specificity-A: 99.34 %
Specificity-B: 99.18 %
Specificity-C: 96.27 %


# Ada Boost Classifier

In [92]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, learning_rate = 2.5395)
abc = abc.fit(X_train_std, y_train)
print''

print 'Predict time'
%time y_pred = abc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score


print ('ClassifyAB accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))


Predict time
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 7.21 ms

ClassifyAB accuracy:---------->82.93 %


# Confusion Matrix for Ada Boost Classifier

In [93]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[52  9  1]
 [11 52  4]
 [ 0 10 66]]
Normalized confusion matrix
[[ 0.84  0.15  0.02]
 [ 0.16  0.78  0.06]
 [ 0.    0.13  0.87]]


In [94]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 82.78 %

Sensitivity-A: 74.29 %
Sensitivity-B: 77.61 %
Sensitivity-C: 86.84 %

Precision-A: 83.63 %
Precision-B: 73.72 %
Precision-C: 91.97 %

Specificity-A: 91.79 %
Specificity-B: 91.44 %
Specificity-C: 96.21 %


# KNeighbors Classifier

In [95]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=2, weights = 'distance', algorithm = 'auto', p = 1)
knc = knc.fit(X_train_std, y_train)
print''

print 'Predict time'
%time y_pred = knc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score



print ('ClassifyKN accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))



Predict time
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.52 ms

ClassifyKN accuracy:---------->99.02 %


# Confusion Matrix for KNeighbors Classifier

In [96]:
from sklearn.metrics import confusion_matrix
import itertools

#%matplotlib inline
class_names = np.array(['normal', 'stair ascent', 'stair descent'])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[61  1  0]
 [ 0 67  0]
 [ 0  1 75]]
Normalized confusion matrix
[[ 0.98  0.02  0.  ]
 [ 0.    1.    0.  ]
 [ 0.    0.01  0.99]]


In [97]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TPa = float(cm[0][0])
Eab = float(cm[0][1])
Eac = float(cm[0][2])
#####################
Eba = float(cm[1][0])
TPb = float(cm[1][1])
Ebc = float(cm[1][2])
#####################
Eca = float(cm[2][0])
Ecb = float(cm[2][1])
TPc = float(cm[2][2])

Pa = TPa/(TPa+Eba+Eca)
Pb = TPb/(TPb+Eab+Ecb)
Pc = TPc/(TPc+Eac+Ebc)

Ra = TPa/(TPa+Eab+Eab)
Rb = TPb/(TPb+Eba+Ebc)
Rc = TPc/(TPc+Eca+Ecb)

TNa = TPb+Ebc+Ecb+TPc
TNb = TPa+Eac+Eca+TPc
TNc = TPa+Eab+Eba+TPb

Sa = TNa/(TNa+Eba+Eca)
Sb = TNb/(TNb+Eab+Eac)
Sc = TNc/(TNc+Eac+Ebc)

Acc = (TPa+TPb+TPc)/(TPa+Eab+Eac+Eba+TPb+Ebc+Eca+Ecb+TPc)

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Accuracy: %.2f %%' % (Acc*100))
print ''
print ('Sensitivity-A: %.2f %%' % (Ra*100))
print ('Sensitivity-B: %.2f %%' % (Rb*100))
print ('Sensitivity-C: %.2f %%' % (Rc*100))
print ''
print ('Precision-A: %.2f %%' % (Pa*100))
print ('Precision-B: %.2f %%' % (Pb*100))
print ('Precision-C: %.2f %%' % (Pc*100))
print ''
print ('Specificity-A: %.2f %%' % (Sa*100))
print ('Specificity-B: %.2f %%' % (Sb*100))
print ('Specificity-C: %.2f %%' % (Sc*100))


PERFORMANCE MEASURES BY CONFUSION MATRIX
Accuracy: 99.02 %

Sensitivity-A: 96.83 %
Sensitivity-B: 100.00 %
Sensitivity-C: 98.68 %

Precision-A: 100.00 %
Precision-B: 97.15 %
Precision-C: 100.00 %

Specificity-A: 100.00 %
Specificity-B: 99.19 %
Specificity-C: 100.00 %


# Classificadores Utilizados:

### knc : KNeighborsClassifier; 
### abc : AdaBoostClassifier; 
### dtc : tree.DecisionTreeClassifier; 
### rfc : RandomForestClassifier; 
### gnb : GaussianNB; 
### rbf_svc : SVMClassifierRBF; 
### poly_svc = SVMClassifierPoly

In [75]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('knc', knc), ('abc', abc), ('dtc', dtc), ('rfc', rfc), ('gnb', gnb), 
                                     ("rbf_svc", rbf_svc), ('poly_svc', poly_svc)], voting='hard',weights=[3,1,1,7,1,4,3])
eclf1 = eclf1.fit(X_train_std, y_train)

NameError: name 'poly_svc' is not defined

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

for clf, label in zip([knc, abc, dtc, rfc, gnb, rbf_svc, poly_svc, eclf1], ['KNieghbors', 'Ada Boost', 'Decision Tree', 
                                                                           'Random Forest', 'Naive Bayes', 'SVM-RBF', 
                                                                           'SVM-Poly', 'Ensemble']):
    scores = (cross_val_score(clf, X_train_std, y_train, cv=5, scoring='accuracy'))*100
    #print("Accuracy: %0.2f %%(+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    print("Accuracy: %0.2f %% [%s]" % (scores.mean(), label))

Accuracy: 96.73 % [KNieghbors]
Accuracy: 77.07 % [Ada Boost]
Accuracy: 91.99 % [Decision Tree]
Accuracy: 98.85 % [Random Forest]
Accuracy: 79.09 % [Naive Bayes]
Accuracy: 96.89 % [SVM-RBF]
Accuracy: 95.92 % [SVM-Poly]
Accuracy: 99.67 % [Ensemble]


In [33]:


import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.


In [34]:
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn import cross_validation

pca = PCA(n_components=10)# adjust yourself
pca.fit(X_train)
X_t_train = pca.transform(X_train)
X_t_test = pca.transform(X_test)

clf = rfc
clf.fit(X_t_train, y_train)
print 'score', clf.score(X_t_test, y_test)

clf.predict(X_t_test)

scores = (cross_val_score(clf, X_train_std, y_train, cv=5, scoring='accuracy'))*100
print("Accuracy: %0.2f %%" % (scores.mean()))

score 0.990243902439
Accuracy: 99.35 %


BAK
import math
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
TP = float(cm[0][0])
FP = float(cm[1][0])
FN = float(cm[0][1])
TN = float(cm[1][1])

print 'PERFORMANCE MEASURES BY CONFUSION MATRIX'
print ('Sensitivity: %.2f %%' % (((TP)/(TP+FN))*100))
print ('Specificity: %.2f %%' % ((TN)/((FP+TN))*100))
print ('Accuracy: %.2f %%' % ((TP+TN)/((TP+FN)+(FP+TN))*100))
print ('phi: %.2f ' % ((TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))
